In [1]:
import torchvision.datasets
import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from torchvision.datasets import ImageFolder
from torch.autograd import Variable

torch.manual_seed(10)

In [2]:
EPOCH = 25
BATCH_SIZE = 4
LR = 0.001

In [3]:
transform = transforms.Compose([   
        transforms.Resize([96, 96]),  
        transforms.ToTensor(),  
        transforms.Normalize(mean=(0,0,0),std=(1,1,1))
])

train_set = ImageFolder('./birds/train/',transform=transform)
train_loader = Data.DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_set = ImageFolder('./birds/test/', transform=transform)
test_loader = Data.DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)

In [4]:
print(train_set.classes)
print(train_set.class_to_idx)
print(train_set.__len__)

print(test_set.classes)
print(test_set.class_to_idx)
print(test_set.__len__)

['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird']
{'001.Black_footed_Albatross': 0, '002.Laysan_Albatross': 1, '003.Sooty_Albatross': 2, '004.Groove_billed_Ani': 3, '005.Crested_Auklet': 4, '006.Least_Auklet': 5, '007.Parakeet_Auklet': 6, '008.Rhinoceros_Auklet': 7, '009.Brewer_Blackbird': 8, '010.Red_winged_Blackbird': 9}
<bound method DatasetFolder.__len__ of Dataset ImageFolder
    Number of datapoints: 543
    Root location: ./birds/train/>
['001.Black_footed_Albatross', '002.Laysan_Albatross', '003.Sooty_Albatross', '004.Groove_billed_Ani', '005.Crested_Auklet', '006.Least_Auklet', '007.Parakeet_Auklet', '008.Rhinoceros_Auklet', '009.Brewer_Blackbird', '010.Red_winged_Blackbird']
{'001.Black_footed_Albatross': 0, '002.Laysan_Albatross': 1, '003.Sooty_Albatross': 2, '004.Groove_billed_Ani':

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential( # 输入数据大小 (3,96,96)
            nn.Conv2d(
                in_channels=3, # 输入的图片层数
                out_channels=16, # 卷积核数量
                kernel_size=5, # 卷积核大小
                stride=1,
                padding=2,
            ), # 输出数据大小 (16,96,96)
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
        )
        self.conv2 = nn.Sequential( # 输入数据大小 (16,48,48)
            nn.Conv2d(16, 32, 5, 1, 2), # 输出数据大小 (32,48,48)
            nn.ReLU(),
            nn.MaxPool2d(2), # 输出数据大小 (32,24,24)
        )
        self.fc1 = nn.Linear(32*24*24, 32*24) # 全连接层
        self.out = nn.Linear(32*24, 10)
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # 展平多维的卷积图
        x = self.fc1(x)
        output = self.out(x)
        return output, x

In [6]:
cnn = CNN()
print(cnn)

CNN(
  (conv1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=18432, out_features=768, bias=True)
  (out): Linear(in_features=768, out_features=10, bias=True)
)


In [7]:
optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss() # 选择损失函数

for epoch in range(EPOCH):
    print('EPOCH ' + str(epoch))
    for step, (b_x, b_y) in enumerate(train_loader):
        output = cnn(b_x)[0]
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if step % 50 == 0:
            correct = 0
            total = 0
            
            for data in test_loader:
                images,labels = data
                outputs = cnn(Variable(images))
                predicted = torch.max(outputs[0], 1).indices
                total += labels.size(0)
                correct += (predicted == labels).sum()

            print('Accuracy of the network on the test images: %d %%' % (100 * correct / total))
            
            
print('End')

Accuracy of the network on the test images: 10 %
Accuracy of the network on the test images: 13 %
Accuracy of the network on the test images: 18 %
Accuracy of the network on the test images: 14 %
Accuracy of the network on the test images: 15 %
Accuracy of the network on the test images: 18 %
Accuracy of the network on the test images: 20 %
Accuracy of the network on the test images: 31 %
Accuracy of the network on the test images: 23 %
Accuracy of the network on the test images: 31 %
Accuracy of the network on the test images: 32 %
Accuracy of the network on the test images: 34 %
Accuracy of the network on the test images: 34 %
Accuracy of the network on the test images: 46 %
Accuracy of the network on the test images: 50 %
Accuracy of the network on the test images: 46 %
Accuracy of the network on the test images: 59 %
Accuracy of the network on the test images: 56 %
Accuracy of the network on the test images: 56 %
Accuracy of the network on the test images: 68 %
Accuracy of the netw